In [ ]:
import psycopg2
import time
import statistics
from datasets import load_dataset
from config import load_config

def create_table():
    cmd = """ CREATE TABLE IF NOT EXISTS bookcorpus (
        sentence_id INTEGER PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
        sentence TEXT NOT NULL
        )"""
    try:
        config = load_config()
        with psycopg2.connect(**config) as conn:
            with conn.cursor() as cur:
                    cur.execute(cmd)
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

    print("Table bookcorpus created.")

def insert_sentences(ds_chunk):
    sql = """INSERT INTO bookcorpus (sentence) VALUES (%s)"""
    text_chunks = []
    for example in ds_chunk:
        text = example["text"]
        text_chunks.append((text,))
    try:
        config = load_config()
        with psycopg2.connect(**config) as conn:
            with conn.cursor() as cur:
                cur.executemany(sql, text_chunks)
            conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

if __name__ == '__main__':
    create_table()
    times = []
    ds = load_dataset("williamkgao/bookcorpus100mb", split="train")
    chunk_size = 10000
    for i in range(0, len(ds), chunk_size):
        start_time = time.time()
        chunk = ds.select(range(i, min(i + chunk_size, len(ds))))
        insert_sentences(chunk)
        end_time = time.time()
        times.append(end_time - start_time)
    
    print("Finished storing textual data.")
    min_time = min(times)
    max_time = max(times)
    avg_time = sum(times) / len(times)
    std_dev = statistics.stdev(times) if len(times) > 1 else 0
    print(f"Minimum time: {min_time:.6f} seconds")
    print(f"Maximum time: {max_time:.6f} seconds")
    print(f"Average time: {avg_time:.6f} seconds")
    print(f"Standard deviation: {std_dev:.6f} seconds")
    

In [ ]:
import psycopg2
import time
import statistics
from sentence_transformers import SentenceTransformer
from config import load_config

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
times = []

def create_embeddings_table():
    cmd = """ CREATE TABLE IF NOT EXISTS embeddings (
        id INTEGER PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
        vec double precision[]
        )"""
    try:
        config = load_config()
        with psycopg2.connect(**config) as conn:
            with conn.cursor() as cur:
                    cur.execute(cmd)
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)
    finally:
        print("Table embeddings created.")

def inserts_to_embeddings():
    chunk_size = 10000
    sql = """SELECT sentence FROM bookcorpus ORDER BY sentence_id ASC;"""
    insert_sql = """INSERT INTO embeddings (vec) VALUES (%s);"""
    count = 0
    try:
        config = load_config()
        with psycopg2.connect(**config) as read_conn:
            read_conn.autocommit = True
            with read_conn.cursor() as read_cursor:
                read_cursor.execute(sql)
                with psycopg2.connect(**config) as write_conn:
                    with write_conn.cursor() as write_cursor:
                        while True:
                            rows = read_cursor.fetchmany(chunk_size)
                            if not rows:
                                break
                            sentences = [row[0] for row in rows]
                            count += len(sentences)
                            embeddings = model.encode(sentences)
                            emb_to_insert = [(embedding.tolist(),) for embedding in embeddings]
                            start_time = time.time()
                            write_cursor.executemany(insert_sql, emb_to_insert)
                            write_conn.commit()
                            end_time = time.time()
                            times.append(end_time - start_time)
                             
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

    print(f"Finished storing the {count} embeddings.")

def performance():
    min_time = min(times)
    max_time = max(times)
    avg_time = sum(times) / len(times)
    std_dev = statistics.stdev(times) if len(times) > 1 else 0
    print(f"Minimum time: {min_time:.6f} seconds")
    print(f"Maximum time: {max_time:.6f} seconds")
    print(f"Average time: {avg_time:.6f} seconds")
    print(f"Standard deviation: {std_dev:.6f} seconds")

if __name__ == '__main__':
    create_embeddings_table()
    inserts_to_embeddings()
    performance()


Table embeddings created.
Finished storing the 1527753 embeddings.
Minimum time: 8.627728 seconds
Maximum time: 11.970323 seconds
Average time: 11.211960 seconds
Standard deviation: 0.323633 seconds


In [25]:
import psycopg2
import math
import time
import statistics
from config import load_config

times = []

def sort_by_manhattan_distance(list_of_lists, reference_list):
    return sorted(list_of_lists, key=lambda item:
        sum(abs(x - y) for x, y in zip(item[1], reference_list))
    )

def sort_by_euclidean_distance(list_of_lists, reference_list):
    return sorted(list_of_lists, key=lambda item: math.sqrt(
        sum((x - y) ** 2 for x, y in zip(item[1], reference_list))
    ))

def query_to_bookcorpus(id):
    sql = """SELECT sentence FROM bookcorpus where sentence_id = (%s);"""
    try:
        config = load_config()
        with psycopg2.connect(**config) as read_conn:
            with read_conn.cursor() as read_cursor:
                read_cursor.execute(sql, (id,))
                res = read_cursor.fetchone()
                return res[0]          
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

def print_similars_sentences(embeddings_ids):
    for id in embeddings_ids:
        print(query_to_bookcorpus(id))

def top2_similars_embeddings(emb_vec):
    sql = """SELECT id, vec FROM embeddings ORDER BY id ASC LIMIT 10000;"""
    try:
        start_time = time.time()
        config = load_config()
        with psycopg2.connect(**config) as read_conn:
            read_conn.autocommit = True
            with read_conn.cursor() as read_cursor:
                read_cursor.execute(sql)
                rows = read_cursor.fetchall()
                
                sorted_euclidean = sort_by_euclidean_distance(rows, emb_vec)            
                top2 = [sorted_euclidean[1][0], sorted_euclidean[2][0]]
                print("Top 2 similars by Euclidean distance:")
                print_similars_sentences(top2)
                sorted_manhattan = sort_by_manhattan_distance(rows, emb_vec)
                top2 = [sorted_manhattan[1][0], sorted_manhattan[2][0]]
                print("Top 2 similars by Manhattan distance:")
                print_similars_sentences(top2)      
                end_time = time.time()
                times.append(end_time - start_time)       
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

def performance():
    min_time = min(times)
    max_time = max(times)
    avg_time = sum(times) / len(times)
    std_dev = statistics.stdev(times) if len(times) > 1 else 0
    print(f"Minimum time: {min_time:.6f} seconds")
    print(f"Maximum time: {max_time:.6f} seconds")
    print(f"Average time: {avg_time:.6f} seconds")
    print(f"Standard deviation: {std_dev:.6f} seconds")

if __name__ == '__main__':
    sentences = []
    for i in range (1, 11):
        sentences.append(query_to_bookcorpus(i))

    for i in range (1, 11):
        print(sentences[i - 1])
        sql = """SELECT id, vec FROM embeddings where id = (%s);"""
        try:
            config = load_config()
            with psycopg2.connect(**config) as read_conn:
                with read_conn.cursor() as read_cursor:
                    read_cursor.execute(sql, (i,))
                    res = read_cursor.fetchone()
                    top2_similars_embeddings(res[1])
        except (psycopg2.DatabaseError, Exception) as error:
            print(error)
        print()
    
    performance()

usually , he would be tearing around the living room , playing with his toys .
Top 2 similars by Euclidean distance:
i 'd watch him when he was n't looking , and paid attention to the little things , like how he shielded my body with his whenever we were going somewhere .
she followed him as he made his way into the living room .
Top 2 similars by Manhattan distance:
i 'd watch him when he was n't looking , and paid attention to the little things , like how he shielded my body with his whenever we were going somewhere .
`` what i 'm wondering is what got him so fired up that he would stomp into a hospital , call you out , and then punch you ?

but just one look at a minion sent him practically catatonic .
Top 2 similars by Euclidean distance:
he could n't fight the warmth that spread through his chest when mason snuggled close to his side .
`` minions . ''
Top 2 similars by Manhattan distance:
his heartbeat accelerated at the thought .
he was staring at aidan and noah with a contemplat